In [52]:
import dataset
from scipy import stats
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TkAgg')

In [53]:
df = pd.read_csv(r"C:\Users\andre\OneDrive\Documente\GitHub\Smartphone dataset price\SmartphonePriceDataset\smartphones.csv")
df['Storage'] = df['Storage'].apply(lambda x: 32 if x < 32 else x)
df.fillna(df.median(numeric_only=True), inplace=True)
bins = [60, 200, 500, 1000, 2200]
labels = ['low', 'medium', 'high', 'premium']
df['Price Category'] = pd.cut(df['Final Price'], bins=bins, labels=labels)
df



,Smartphone,Brand,Model,RAM,Storage,Color,Free,Final Price,Price Category
0,Realme C55 8/256GB Sunshower Libre,Realme,C55,8.0,256.0,Yellow,Yes,231.60,medium
1,Samsung Galaxy M23 5G 4/128GB Azul Libre,Samsung,Galaxy M23,4.0,128.0,Blue,Yes,279.00,medium
2,Motorola Moto G13 4/128GB Azul Lavanda Libre,Motorola,Moto G13,4.0,128.0,Blue,Yes,179.01,low
3,Xiaomi Redmi Note 11S 6/128GB Gris Libre,Xiaomi,Redmi Note 11S,6.0,128.0,Gray,Yes,279.99,medium
4,Nothing Phone (2) 12/512GB Blanco Libre,Nothing,Phone (2),12.0,512.0,White,Yes,799.00,high
...,...,...,...,...,...,...,...,...,...
1811,Xiaomi Redmi Note 8 4/64GB Azul Libre,Xiaomi,Redmi Note 8,4.0,64.0,Blue,Yes,249.01,medium
1812,Xiaomi Redmi Note 8T 4/128GB Azul Estelar Libre,Xiaomi,Redmi Note 8T,4.0,128.0,Blue,Yes,200.00,low
1813,Xiaomi Redmi Note 9 4/128GB Blanco Libre,Xiaomi,Redmi Note 9,4.0,128.0,White,Yes,269.00,medium
1814,Xiaomi Redmi Note 9S 4/64GB Dual SIM Gris Libre,Xiaomi,Note 9S,4.0,64.0,Gray,Yes,211.00,medium


In [54]:
Q1 = df['Final Price'].quantile(0.25)
Q3 = df['Final Price'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df = df[(df['Final Price'] >= lower_bound) & (df['Final Price'] <= upper_bound)]


In [55]:
text = '''Datasetul este format din 1816 randuri si este pe 9 coloane:
1. Numele Smartphone-ului
2. Modelul
3. RAM (Memorie cu Acces Aleator)
4. Spatiul de Stocare
5. Culoare
6. Free (indica daca are sau nu un contract cu o companie de telefonie mobila)
7. Pretul smartphone-ului
8. Categorie de Pret : consta luxul telefonului bazat pe intru-n interval de pret
idei-------------------:Numar de vanzari'''

print(text)


Datasetul este format din 1816 randuri si este pe 9 coloane:
1. Numele Smartphone-ului
2. Modelul
3. RAM (Memorie cu Acces Aleator)
4. Spatiul de Stocare
5. Culoare
6. Free (indica daca are sau nu un contract cu o companie de telefonie mobila)
7. Pretul smartphone-ului
8. Categorie de Pret : consta luxul telefonului bazat pe intru-n interval de pret
idei-------------------:Numar de vanzari


In [56]:
print(f"Număr de rânduri: {df.shape[0]}")
print(f"Număr de coloane: {df.shape[1]}")

Număr de rânduri: 1722
Număr de coloane: 9


In [57]:
print("Coloanele datasetului sunt:")
print(df.columns)
print("\n")
print("Tipurile de dataset-ului :\n")
print(df.dtypes)
print("\n")
print(df.describe())
print("\n")

print("Valori Not A Number : ")
print(df.isnull().sum())
print(df.describe())


Coloanele datasetului sunt:
Index(['Smartphone', 'Brand', 'Model', 'RAM', 'Storage', 'Color', 'Free',
       'Final Price', 'Price Category'],
      dtype='object')


Tipurile de dataset-ului :

Smartphone          object
Brand               object
Model               object
RAM                float64
Storage            float64
Color               object
Free                object
Final Price        float64
Price Category    category
dtype: object


               RAM      Storage  Final Price
count  1722.000000  1722.000000  1722.000000
mean      5.901278   145.119628   430.052178
std       2.257046   101.003683   300.486656
min       1.000000    32.000000    60.460000
25%       4.000000    64.000000   199.000000
50%       6.000000   128.000000   327.990000
75%       8.000000   256.000000   589.000000
max      12.000000  1000.000000  1326.030000


Valori Not A Number : 
Smartphone        0
Brand             0
Model             0
RAM               0
Storage           0
Color           

In [58]:
plt.figure(figsize=(8, 6))
df['Final Price'].hist(bins=30, edgecolor='black')
plt.title('Distributia Preturilor')
plt.xlabel('Final Price')
plt.ylabel('Frecventa')
plt.show(block=True)


In [59]:
def gini_index(data, target_column):
    class_counts = data[target_column].value_counts(normalize=True)
    gini = 1 - sum(class_counts**2)
    return gini

gini = gini_index(df, 'Price Category')
print(f"Gini Index pentru 'Price Category': {gini}")

Gini Index pentru 'Price Category': 0.6860192817956055


In [60]:
def gini_for_attribute(data, attribute, target_column):
    grouped = data.groupby(attribute)
    gini_values = {}

    for group, subset in grouped:
        gini_values[group] = gini_index(subset, target_column)

    return gini_values

gini_values_for_brand = gini_for_attribute(df, 'Brand', 'Price Category')
print("Gini Index pentru fiecare brand:", gini_values_for_brand)

Gini Index pentru fiecare brand: {'Alcatel': 0.24489795918367352, 'Apple': 0.7097875828153815, 'Asus': 0.6666666666666667, 'BQ': 0.0, 'Blackview': 0.49382716049382713, 'CAT': 0.6111111111111112, 'Crosscall': 0.48979591836734704, 'Cubot': 0.4377162629757785, 'Doro': 0.4444444444444444, 'Fairphone': 0.0, 'Funker': 0.0, 'Gigaset': 0.4444444444444444, 'Google': 0.6666666666666667, 'Hammer': 0.5804988662131519, 'Honor': 0.470414201183432, 'Huawei': 0.5921822099107419, 'LG': 0.0, 'Lenovo': 0.0, 'Maxcom': 0.0, 'Microsoft': 0.0, 'Motorola': 0.6531240381655894, 'Nokia': 0.5416666666666666, 'Nothing': 0.19753086419753085, 'OPPO': 0.6304347826086957, 'OnePlus': 0.48979591836734704, 'POCO': 0.4219202494987747, 'Qubo': 0.0, 'Realme': 0.5785667324128863, 'SPC': 0.0, 'Samsung': 0.6931721737743577, 'Sony': 0.5, 'Swissvoice': 0.0, 'TCL': 0.4521604938271605, 'Ulefone': 0.5711111111111111, 'Vivo': 0.5898491083676268, 'Xiaomi': 0.6021461235950445, 'ZTE': 0.0}


In [61]:
import numpy as np

def entropy(data):
    class_counts = data['Price Category'].value_counts(normalize=True)

    class_counts = class_counts[class_counts > 0]
    return -np.sum(class_counts * np.log2(class_counts))

parent_entropy = entropy(df)
print(f"Entropia pentru întregul set de date: {parent_entropy}")


Entropia pentru întregul set de date: 1.7928240994534357


In [62]:
# Functie pentru calcularea Information Gain pe baza unui atribut
def information_gain(data, attribute):
    parent_entropy = entropy(data)

    grouped = data.groupby(attribute)
    weighted_entropy = 0
    for group, subset in grouped:
        weighted_entropy += (len(subset) / len(data)) * entropy(subset)
    return parent_entropy - weighted_entropy

info_gain = information_gain(df, 'Brand')
print(f"Information Gain pentru 'Brand': {info_gain}")
info_gain = information_gain(df, 'Storage')
print(f"Information Gain pentru 'Storage': {info_gain}")
info_gain = information_gain(df, 'RAM')
print(f"Information Gain pentru 'RAM': {info_gain}")
info_gain = information_gain(df, 'Brand')
print(f"Information Gain pentru 'Brand': {info_gain}")

Information Gain pentru 'Brand': 0.26251049121179326
Information Gain pentru 'Storage': 0.3982473850517898
Information Gain pentru 'RAM': 0.42673224769343276
Information Gain pentru 'Brand': 0.26251049121179326


In [63]:
# Calcularea corelatiei dintre 'Final Price' și 'RAM','Storage'
correlation = df['Final Price'].corr(df['Storage'])
print(f"Corelatia dintre 'Final Price' si 'Storage': {correlation}")
correlation = df['Final Price'].corr(df['RAM'])
print(f"Corelatia dintre 'Final Price' si 'RAM': {correlation}")

Corelatia dintre 'Final Price' si 'Storage': 0.5821433839604104
Corelatia dintre 'Final Price' si 'RAM': 0.4946746366627295


In [64]:
numeric_columns = df.select_dtypes(include=['number'])
numeric_columns

,RAM,Storage,Final Price
0,8.0,256.0,231.60
1,4.0,128.0,279.00
2,4.0,128.0,179.01
3,6.0,128.0,279.99
4,12.0,512.0,799.00
...,...,...,...
1811,4.0,64.0,249.01
1812,4.0,128.0,200.00
1813,4.0,128.0,269.00
1814,4.0,64.0,211.00


In [65]:
correlation_matrix = numeric_columns.corr()

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()